******

In [3]:
library(tidyverse)
library(readxl)
library(ggplot2)
library(dplyr)

In [5]:
cereal <- read_excel("data/cereal.xlsx")
cereal_arranged <- cereal |>
    select(name, calories, protein, fat, sodium, fiber, carbo, sugars, potass, rating) |>
    arrange(rating)
cereal_arranged

name,calories,protein,fat,sodium,fiber,carbo,sugars,potass,rating
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cap'n'Crunch,120,1,2,220,0.0,12.0,12,35,18.04285
Cinnamon Toast Crunch,120,1,3,210,0.0,13.0,9,45,19.82357
Honey Graham Ohs,120,1,2,220,1.0,12.0,11,45,21.87129
Count Chocula,110,1,1,180,0.0,12.0,13,65,22.39651
Cocoa Puffs,110,1,1,180,0.0,12.0,13,55,22.73645
Golden Grahams,110,1,1,280,0.0,15.0,9,45,23.80404
Lucky Charms,110,2,1,180,0.0,12.0,12,55,26.73451
Trix,110,1,1,140,0.0,13.0,12,25,27.75330
Fruity Pebbles,110,1,1,135,0.0,13.0,12,25,28.02576
